# code to run this pipeline on a local pc

In [1]:
import sys
import os

import utils

from preprocess import main as preprocess_nd2

# 1. PREPROCESS

In [2]:

DATA_PTH = r'C:\Users\munib\POSTDOC\DATA\g5-HT-free\albert-g5ht-example\022025_eft_41z_starved_worm002'
INPUT_ND2 = '022025_eft_41z_starved_worm002.nd2'
INPUT_ND2_PTH = os.path.join(DATA_PTH, INPUT_ND2)
NOISE_PTH = r'C:\Users\munib\POSTDOC\CODE\g5-HT3_imaging_pipeline/noise_042925.tif'

OUT_DIR = os.path.splitext(INPUT_ND2_PTH)[0]

STACK_LENGTH = 41

noise_stack = utils.get_noise_stack(NOISE_PTH, STACK_LENGTH)
stack_range = utils.get_range_from_nd2(INPUT_ND2_PTH) # aka frame indices

In [3]:
# Simulate command-line arguments
sys.argv = ["", INPUT_ND2_PTH, "0", NOISE_PTH]

# Call the main function
preprocess_nd2()

Stack 0000 shear corrected
Stack 0000 aligned
